In [ ]:
import json
import numpy as np
import pandas as pd
from pygoko import CoverTree
with open("graphistry_creds.json") as creds:
    creds = json.load(creds)
import graphistry
graphistry.register(api=3, username=creds["username"], password=creds["password"])

import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.int64)
x_train = x_train.reshape(-1, 28*28)

In [ ]:
tree = CoverTree()
tree.set_leaf_cutoff(50)
tree.set_scale_base(1.3)
tree.set_min_res_index(-20)
tree.fit(x_train,y_train)

In [ ]:
source = []
destination = []
weight = []
depth = []

node_colors =  [0xFF000000, 0xFFFF0000, 0xFFFFFF00, 0x00FF0000, 0x0000FF00, 0xFF00FF00, 0x88000000, 0x88888800, 0x00880000, 0x00008800]

nodes_centers = set()

unvisited_nodes = [tree.root()]
while len(unvisited_nodes) > 0: 
    node = unvisited_nodes.pop()
    nodes_centers.add(node.address()[1])
    for child in node.children():
        source.append(node.address()[1])
        destination.append(child.address()[1])
        depth.append(node.address()[0] * 0xFF000000)
        weight.append(1.3**(-node.address()[0]))
        unvisited_nodes.append(child)

edges = pd.DataFrame({
    'source': source,
    'destination': destination,
    'weight': weight,
    'depth': depth,
})

node_id = list(nodes_centers)
label = [node_colors[y_train[i]] for i in node_id]

nodes = pd.DataFrame({
    "node_id" : node_id,
    "label" : label
})

In [ ]:
URL_PARAMS = {'play': 5000, 'edgeCurvature': 0.1, 'precisionVsSpeed': -3}
g = graphistry.nodes(nodes).edges(edges).bind(source='source', destination='destination', node="node_id", point_color="label").settings(url_params=URL_PARAMS)

In [ ]:
g.bind(edge_weight='weight').settings(url_params={**URL_PARAMS, 'expansionRatio': 40, 'edgeInfluence': 7}).plot(render=True)